# Sección B: Quién es Quién en los Precios

## Introducción

La siguiente asignación está referida al análisis de la base de datos de la Procuradoría Federal del Consumidor PROFECO, del programa Quien es Quién (QQP) que recaba y difunde información de precios de productos de consumo regular en el hogar, como alimentos, bebidas, productos de aseo personal y del hogar, medicinas, electrodomésticos y artículos de temporada para ofrecer información que permite tomar decisiones de compra mediante la comparación de precios. [1]

Esta base de datos es un registro histórico diario de más de 2,000 productos, a partir de 2015, en diversos establecimientos de la República Mexicana. Se pide realizar el análisis con PySpark SQL.

In [1]:
# Step 0. Cargar bibliotecas y funciones ·······················#
import pyspark
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import findspark
findspark.init()

from pyspark.sql import SQLContext, SparkSession
from pyspark import SparkContext

In [2]:
# Step 1. Cargar los datos del archivo CSV ·····················#
# 1.1 Creamos primero un contexto
name = 'seccion b'
sc = SparkContext(appName = name)
sql_c = SQLContext(sc)
# 1.2 Y luego pasamos a leer el csv
df = sql_c.read.format('csv').options(header='true').load('all_data.csv')
sql_c.registerDataFrameAsTable(df, "my_table")
# 1.3 Es conveniente dar un vistazo a como está definido el esquema de la tabla que hemos generado
df.printSchema()

root
 |-- producto: string (nullable = true)
 |-- presentacion: string (nullable = true)
 |-- marca: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- catalogo: string (nullable = true)
 |-- precio: string (nullable = true)
 |-- fechaRegistro: string (nullable = true)
 |-- cadenaComercial: string (nullable = true)
 |-- giro: string (nullable = true)
 |-- nombreComercial: string (nullable = true)
 |-- direccion: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- latitud: string (nullable = true)
 |-- longitud: string (nullable = true)



In [3]:
# Step 2. Creamos la vista temporal de la tabla ················#
df.createOrReplaceTempView("my_table")
# Damos una primera vista a los datos
sqlDF = sql_c.sql("SELECT * FROM my_table LIMIT 5")
sqlDF.show()
# Y pasamos a responder las preguntas

+--------------------+--------------------+--------+----------------+----------------+------+--------------------+------------------+----------+--------------------+--------------------+----------------+--------------------+--------+----------+
|            producto|        presentacion|   marca|       categoria|        catalogo|precio|       fechaRegistro|   cadenaComercial|      giro|     nombreComercial|           direccion|          estado|           municipio| latitud|  longitud|
+--------------------+--------------------+--------+----------------+----------------+------+--------------------+------------------+----------+--------------------+--------------------+----------------+--------------------+--------+----------+
|CUADERNO FORMA IT...|96 HOJAS PASTA DU...|ESTRELLA|MATERIAL ESCOLAR|UTILES ESCOLARES|  25.9|2011-05-18 00:00:...|ABASTECEDORA LUMEN|PAPELERIAS|ABASTECEDORA LUME...|CANNES No. 6 ESQ....|DISTRITO FEDERAL|TLALPAN          ...|19.29699|-99.125417|
|            CRAYONE

## 1. Procesamiento de los datos

### a. ¿Cuántos registros hay?

In [4]:
sqlDF = sql_c.sql("SELECT COUNT(*) FROM my_table")
sqlDF.show(2,False)

+--------+
|count(1)|
+--------+
|62530715|
+--------+



Respuesta: Hay 62,530,715 registros. Estos más de 62.5 millones de registros están dentro de la tabla creada con 15 variables.

### b. ¿Cuántas categorías?

In [5]:
sqlDF = sql_c.sql("SELECT COUNT(DISTINCT categoria) FROM my_table")
sqlDF.show(2,False)

+-------------------------+
|count(DISTINCT categoria)|
+-------------------------+
|41                       |
+-------------------------+



Respuesta: Hay 41 categorías en la variable "categoría".

### c.¿Cuántas cadenas comerciales están siendo monitoreadas?

In [6]:
sqlDF = sql_c.sql("SELECT COUNT(DISTINCT cadenaComercial) FROM my_table")
sqlDF.show(2,False)

+-------------------------------+
|count(DISTINCT cadenaComercial)|
+-------------------------------+
|705                            |
+-------------------------------+



Se están monitoreando 705 cadenas comerciales.

### d.¿Cómo podrías determinar la calidad de los datos? ¿Detectaste algún tipo de inconsistencia o error en la fuente?

Para iniciar, al haber hecho el vistazo del esquema de la tabla, pude observar que los precios están siendo tratados como una cadena (string), en vez de ser una variable numérica. De igual manera la fecha de registro, latitud y longitud se tratan como cadena. 

### e.¿Cuáles son los productos más monitoreados en cada entidad?

In [17]:
# Si entidad se refiere a estados, hagamos una lista de los mismos
sqlDF = sql_c.sql("SELECT DISTINCT estado FROM my_table WHERE estado IS NOT NULL")
sqlDF.show()

+--------------------+
|              estado|
+--------------------+
|        QUINTANA ROO|
|          NUEVO LEÓN|
|             SINALOA|
|             TABASCO|
|     BAJA CALIFORNIA|
|            TLAXCALA|
|COAHUILA DE ZARAGOZA|
|       ESQ. SUR 125"|
|             CHIAPAS|
| COL. EDUARDO GUERRA|
|VERACRUZ DE IGNAC...|
|              SONORA|
|             YUCATÁN|
| MICHOACÁN DE OCAMPO|
|             DURANGO|
|            GUERRERO|
|             NAYARIT|
|           CHIHUAHUA|
|    DISTRITO FEDERAL|
|              estado|
+--------------------+
only showing top 20 rows



Aquí podemos observar unos detalles como que el campo "estado" contiene no solo estados de México sino tambien otras entidades como colonias (Col. Eduardo Guerra) o calles (Esq. Sur 125"). Esto significa que la base requiere cierta limpieza de datos como se indica en el inciso d, referido a la calidad. En si, México tiene 31 estados y la capital.[2]

In [22]:
# Dado que hay datos inconsistentes, vamos a mostrar 4 estados 
sqlDF = sql_c.sql("WITH estado_products AS(\
SELECT estado, producto, COUNT(*) product_count, \
ROW_NUMBER() OVER (PARTITION BY estado ORDER BY COUNT(*) DESC) rn \
FROM my_table WHERE estado IS NOT NULL AND estado IN ('NUEVO LEÓN','SINALOA','TABASCO','DISTRITO FEDERAL') \
GROUP BY estado, producto) \
SELECT estado, producto, product_count FROM estado_products WHERE rn<=3")
sqlDF.show()

+----------------+--------------------+-------------+
|          estado|            producto|product_count|
+----------------+--------------------+-------------+
|      NUEVO LEÓN|   DETERGENTE P/ROPA|        50307|
|      NUEVO LEÓN|            REFRESCO|        49592|
|      NUEVO LEÓN|LECHE ULTRAPASTEU...|        43803|
|         SINALOA|            REFRESCO|        33115|
|         SINALOA|   DETERGENTE P/ROPA|        27177|
|         SINALOA|             SHAMPOO|        22435|
|         TABASCO|            REFRESCO|        28754|
|         TABASCO|   DETERGENTE P/ROPA|        26431|
|         TABASCO|           LAVADORAS|        26361|
|DISTRITO FEDERAL|            REFRESCO|       287463|
|DISTRITO FEDERAL|                 FUD|       207569|
|DISTRITO FEDERAL|LECHE ULTRAPASTEU...|       175640|
+----------------+--------------------+-------------+



Sobre esta pregunta, vamos a asumir que entidad se refiere al estado. Podemos observar que uno de los productos top comunes del estudio de precios en Nuevo León, Sinaloa y Tabasco es el detergente sin embargo en el distrito federal es el refresco. 

### f. ¿Cuál es la cadena comercial con mayor variedad de productos monitoreados?

In [8]:
sqlDF = sql_c.sql("SELECT cadenaComercial, COUNT(DISTINCT producto) AS counts \
FROM my_table GROUP BY cadenaComercial ORDER BY counts DESC LIMIT 10")
sqlDF.show()

+--------------------+------+
|     cadenaComercial|counts|
+--------------------+------+
|             SORIANA|  1059|
|            WAL-MART|  1051|
|MEGA COMERCIAL ME...|  1049|
|  COMERCIAL MEXICANA|  1036|
|            CHEDRAUI|  1026|
|     MERCADO SORIANA|  1024|
|      BODEGA AURRERA|  1012|
|HIPERMERCADO SORIANA|  1006|
|              H.E.B.|  1001|
|        SORIANA PLUS|   999|
+--------------------+------+



La cadena comercial con mayor variedad de productos monitoreados es Soriana.

## 2. Análisis Exploratorio

### a. Genera una canasta de productos básicos que te permita comparar los precios geográfica y temporalmente. Justifica tu elección y procedimiento

Como productos de canasta de productos básicos tomaría los diez productos más monitoreados, ya que al ver la lista esta contiene alimentos (refrescos, leche, mayonesa, yoghurt, chiles en lata y FUD), artículos personales (desodorante, jabón y shampoo), así como de limpieza (detergente). Este tipo de productos es independiente de la región o del sector socioeconómico. Una vista de SQL permitirá tener disponible los datos para posterior análisis.

In [19]:
# Listaremos entonces los 10 productos más vendidos
sqlDF = sql_c.sql("SELECT producto, COUNT(*) mycount FROM my_table GROUP BY producto ORDER BY mycount DESC LIMIT 10")
sqlDF.show()

+--------------------+-------+
|            producto|mycount|
+--------------------+-------+
|            REFRESCO|1247981|
|   DETERGENTE P/ROPA| 990122|
|                 FUD| 933410|
|LECHE ULTRAPASTEU...| 886716|
|             SHAMPOO| 745467|
|    JABON DE TOCADOR| 744914|
|      CHILES EN LATA| 724862|
|            MAYONESA| 697586|
|             YOGHURT| 632362|
|         DESODORANTE| 623684|
+--------------------+-------+



Podemos observar que los productos más monitoreados son los refrescos y detergentes. El caso de FUD puede referirse a una marca que encontré referida a carnes curadas y quesos en México.

### b. ¿Cuál es la ciudad más cara del país? ¿Cuál es la más barata?

In [12]:
# Aquí vamos a agregar por cada municipio el precio máximo de cada producto de nuestra canasta
sqlDF = sql_c.sql("SELECT DISTINCT municipio, producto, AVG(CAST(precio AS NUMERIC)) OVER \
(PARTITION BY municipio, producto) AS precio_max FROM my_table \
WHERE producto IN ('REFRESCO', 'DETERGENTE P/ROPA', 'FUD', 'LECHE ULTRAPASTEURIZADA', \
'SHAMPOO', 'JABON DE TOCADOR', 'CHILES EN LATA', 'MAYONESA', 'YOGHURT', 'DESODORANTE') \
ORDER BY precio_max DESC;")
sqlDF.createOrReplaceTempView("precio_max_table")

# Aquí vamos a agregar por cada municipio el precio máximo de cada producto de nuestra canasta
sqlDF = sql_c.sql("SELECT DISTINCT municipio, producto, AVG(CAST(precio AS NUMERIC)) OVER \
(PARTITION BY municipio, producto) AS precio_min FROM my_table \
WHERE producto IN ('REFRESCO', 'DETERGENTE P/ROPA', 'FUD', 'LECHE ULTRAPASTEURIZADA', \
'SHAMPOO', 'JABON DE TOCADOR', 'CHILES EN LATA', 'MAYONESA', 'YOGHURT', 'DESODORANTE') \
ORDER BY precio_min ASC;")
sqlDF.createOrReplaceTempView("precio_min_table")

sqlDF = sql_c.sql("SELECT DISTINCT municipio, SUM(precio_max) OVER (PARTITION BY municipio) AS compra_max \
FROM precio_max_table ORDER BY compra_max DESC")
sqlDF.show()

sqlDF = sql_c.sql("SELECT DISTINCT municipio, SUM(precio_min) OVER (PARTITION BY municipio) AS compra_min \
FROM precio_min_table ORDER BY compra_min ASC")
sqlDF.show()

+--------------------+----------+
|           municipio|compra_max|
+--------------------+----------+
|        HUIXQUILUCAN|  309.5615|
|SAN PEDRO GARZA G...|  306.5021|
|         TLAQUEPAQUE|  306.1160|
|     PACHUCA DE SOTO|  304.6828|
|             URUAPAN|  301.6524|
|             REYNOSA|  299.7344|
|              LA PAZ|  299.7268|
|          CUAUTITLÁN|  299.6256|
|        BOCA DEL RÍO|  299.3666|
|             TIJUANA|  298.6934|
|        AZCAPOTZALCO|  298.1347|
|      ÁLVARO OBREGÓN|  298.0312|
|               TEPIC|  297.6305|
|SAN NICOLÁS DE LO...|  296.8898|
|             TLALPAN|  296.5949|
|         SOLIDARIDAD|  295.7222|
|            COACALCO|  295.6381|
|               LERMA|  295.2396|
|             ORIZABA|  295.1102|
|             ZAPOPAN|  294.5934|
+--------------------+----------+
only showing top 20 rows

+--------------------+----------+
|           municipio|compra_min|
+--------------------+----------+
|TLAJOMULCO DE ZUÑ...|   13.0000|
|CUAJIMALPA DE MOR...|

Respecto a la canasta más cara, podemos decir que es en el municipio de Huixquilucan, con un valor medio sobre todo el periodo de precios de 309.56, mientras que el mínimo debe ser analizado con cuidado, ya que un valor extremo bajo como el de Tlajomulco de Zúñiga puede deberse a que la canasta está incompleta. El valor más bajo razonablemente sería elde Sana Cruz Xoxocotlán,a 241.02. 

## Referencias
 
[1] Gobierno de México - Profeco. (9 de mayo del 2021). *Quién es Quién en los Precios*. Descargado de https://www.profeco.gob.mx/precios/canasta/default.aspx  
